In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '../../python-tools'))
	print(os.getcwd())
except:
	pass


 ### 1. Changing string representation
 * To change the string representation of an instance, define the **__str__()** and ** __repr__()** methods.

In [1]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    # The format code {0.x} specifies the x-attribute of argument 0.
    # So, in the following function, the 0 is actually the instance self
    def __repr__(self):
        return 'Pair({0.x!r}, {0.y!r})'.format(self)

    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)


In [2]:
p = Pair(3,4)
p   # __repr__() output


Pair(3, 4)

In [3]:
print(p)    # __str__() output


(3, 4)


In [4]:
# the special !r formatting code indicates that the output of __repr__()
# should be used instead of __str__(), the default
print('p is {0!r}'.format(p))
print('p is {0}'.format(p))


p is Pair(3, 4)
p is (3, 4)


In [5]:
# alternative way to this implementation, use the % operator as follows:
def __repr__(self):
    return 'Pair(%r, %r)' % (self.x, self.y)


 ### 2. Customizing string formatting
 * define the __format__() method on a class

In [6]:
_formats = {
    'ymd' : '{d.year}-{d.month}-{d.day}',
    'mdy' : '{d.month}/{d.day}/{d.year}',
    'dmy' : '{d.day}/{d.month}/{d.year}'
}


In [7]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)



In [8]:
d = Date(2012, 12, 21)
format(d)


'2012-12-21'

In [9]:
format(d, 'mdy')


'12/21/2012'

In [10]:
'The date is {:ymd}'.format(d)


'The date is 2012-12-21'

In [11]:
'The date is {:mdy}'.format(d)


'The date is 12/21/2012'

In [12]:
from datetime import date


In [13]:
d = date(2019, 10, 1)
format(d)


'2019-10-01'

In [14]:
format(d,'%A, %B %d, %Y')


'Tuesday, October 01, 2019'

In [15]:
'The end is {:%d %b %Y}.'.format(d)


'The end is 01 Oct 2019.'

 ### 3. Saving memory when creating a large number of instances
 * adding the __slots__ attribute to the class definition

In [16]:
# Attribute names listed in the __slots__ specifier are internally mapped to
# specific indices within this array.
class Date:
    __slots__ = ['year', 'month', 'day']
    def __init__(self, year, month, day):
            self.year = year
            self.month = month
            self.day = day


 ### 4. Customizing access to an attribute

In [17]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name

    # getter function
    @property
    def first_name(self):
        return self._first_name

    # setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

    # delete function
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute")


In [18]:
a = Person('Mike')


In [19]:
a.first_name    # Calls the getter


'Mike'

In [20]:
a.first_name = 42   # Calls the setter


TypeError: Expected a string

In [21]:
del a.first_name


AttributeError: Can't delete attribute

 * Properties can also be defined for existing get and set methods.

In [22]:
class Person:
    def __init__(self, first_name):
            self.set_first_name(first_name)

    # Getter function
    def get_first_name(self):
        return self._first_name

    # Setter function
    def set_first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

    # Deleter function (optional)
    def del_first_name(self):
        raise AttributeError("Can't delete attribute")

    # Make a property from existing get/set methods
    first_name = property(get_first_name, set_first_name, del_first_name)


In [23]:
# to inspect a class with a property, we can find the raw methods in the fget,
# fset, and fdel attributes of the property itself.
# Normally, we wouldn’t call fget or fset directly, but they are triggered
# automatically when the property is accessed.
Person.first_name.fget

<function __main__.Person.get_first_name(self)>

In [24]:
import math

In [25]:
# Properties can also be a way to define computed attributes. These are attributes
# that are not actually stored, but computed on demand.
class Circle:
    def __init__(self, radius):
        self.radius = radius
    @property
    def area(self):
        return math.pi * self.radius ** 2
    @property
    def perimeter(self):
        return 2 * math.pi * self.radius


In [26]:
c = Circle(4.0)
c.radius


4.0

In [27]:
c.area   #notice lack of ()


50.26548245743669

In [28]:
c.perimeter     #notice lack of ()


25.132741228718345

 ### 5. Calling method on parent class

In [29]:
class A:
    def spam(self):
        print('A.spam')

class B(A):
    def spam(self):
        print('B.spam')
        super().spam()      # Call parent spam()


In [30]:
# very common use of super() is in the handling of the __init__() method to make
# sure that parents are properly initialized
class A:
    def __init__(self):
        self.x = 0

class B(A):
    def __init__(self):
        super().__init__()
        self.y = 1

In [31]:
# another common use of super() is in code that overrides any of Python’s
# special methods
class Proxy:
    def __init__(self, obj):
        self._obj = obj

    # Delegate attribute lookup to internal obj
    def __getattr__(self, name):
        return getattr(self._obj, name)

    # Delegate attribute assignment
    def __setattr__(self, name, value):
        if name.startswith('_'):
            super().__setattr__(name, value)    #call original __setattr__
        else:
            setattr(self._obj, name, value)


 ### 6. Creating a new kind of class or instance attribute
 * create an entirely new kind of instance attribute, define its functionality in the form of a descriptor class.
 * Descriptors provide the underlying magic for most of Python’s class features, including @classmethod, @staticmethod, @property, and even the __slots__ specification.

In [32]:
# descriptor attribute for an integer type-checked attribute
class Integer:
    def __init__(self, name):
        self.name = name

    def __get__(self, instance, name):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('expected an int')
        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        del instance.__dict__[self.name]


In [33]:
# To use a descriptor, instances of the descriptor are placed into a class
# definition as class variables.
class Point:
    x = Integer('x')
    y = Integer('y')
    def __init__(self, x, y):
        self.x = x
        self.y = y


In [35]:
p = Point(2,3)
p.x     # Calls Point.x.__get__(p,Point)


2

In [36]:
p.y = 5   # call Point.y__set__(p, 5)


In [37]:
p.x = 2.5   # call Point.x__set__(p, 2.5)



TypeError: expected an int

In [38]:
# descriptor for a type-checked attribute
class Typed:
    def __init__(self, name, expected_type):
        self.name = name
        self.expected_type = expected_type

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('Expected '+str(self.expected_type))
        instance.__dict__[self.name] = value

    def delete(self, instance):
        del instance.__dict__[self.name]

In [39]:
# Class decorator that applies it to selected attributes
def typeassert(**kwargs):
    def decorate(cls):
        for name, expected_type in kwargs.items():
            # Attach a Typed descriptor to the class
            setattr(cls, name, Typed(name, expected_type))
        return cls
    return decorate
    

In [40]:
# Example use
@typeassert(name=str, shares=int, price=float)
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

 ### 7. Using lazily computed properties

In [46]:
class lazyproperty:
    def __init__(self, func):
        self.func = func
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value


In [48]:
# utilize this code
class Circle:
    def __init__(self, radius):
        self.radius = radius
    @lazyproperty
    def area(self):
        print('computing area:')
        return math.pi * self.radius ** 2

    @lazyproperty
    def perimeter(self):
        print('computing perimeter:')
        return 2 * math.pi * self.radius


In [49]:
c = Circle(4.0)
c.radius

4.0

In [50]:
c.area


computing area:


50.26548245743669

In [51]:
c.area


50.26548245743669

In [52]:
c.perimeter

computing perimeter:


25.132741228718345

In [53]:
vars(c)

{'radius': 4.0, 'area': 50.26548245743669, 'perimeter': 25.132741228718345}

In [54]:
del c.area
vars(c)

{'radius': 4.0, 'perimeter': 25.132741228718345}

In [55]:
## Delete the variable and see property trigger again
c.area

computing area:


50.26548245743669

 ### 8.Simplifying the initialization of data structures
 * generalize the initialization of data structures into a single `__init__()` function defined in a common base class.

In [56]:
class Structure:
# Class variable that specifies expected fields
    _fields = []
    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))
        # set the arguments
        for name, value in zip(self._fields, args):
            setattr(self, name, value)
            